In [ ]:
def test_stock_price_preprocessing_for_eval(test_stock_prices_df):
    """スコア計算用のstock_pricesの前処理"""
    
    test_stock_prices_df['Date'] = pd.to_datetime(test_stock_prices_df['Date'])
    test_stock_prices_df = test_stock_prices_df[['Date', 'SecuritiesCode', 'Target']]
    
    return test_stock_prices_df

def submission_preprocessin_for_eval(submission_df):
    """スコア計算用のsubmission_dfの前処理"""
    
    submission_df['Date'] = pd.to_datetime(submission_df['Date'])
    
    return submission_df

def calc_spread_return_per_day(day_pred, day_test):
    """当該日付分だけ抽出した状態で、当該日付のR_dayを計算"""
    
    day_test = day_test.merge(right=day_pred, on='SecuritiesCode')
    day_test.sort_values('Rank', inplace=True)
    
    day_S_up = (day_test[:200]['Target'].values * weights).sum() / weights_mean
    day_S_down = (day_test[-200:]['Target'].values[::-1] * weights).sum() / weights_mean
    day_R = day_S_up - day_S_down
    return day_R

In [ ]:
#日付を連続的に生成する。

from datetime import datetime
from datetime import timedelta

start = datetime(2021, 12, 6)
end = datetime(2022, 2, 28)

def daterange(_start, _end):
    """
    日付を連続的に生成するiterator
    
    Parameters
    ----------
    _start : datetime.datetime
        この日からスタートさせたい(iterにはこの日も含む)
    _end : datetime.datetime
        この日までiterに含めたい
        
    Returns
    ----------
    iteratorが返ります。
    
    """
    for n in range((_end - _start).days + 1):
        yield _start + timedelta(n)



In [ ]:
# #./data/supplemental_files/stock_prices.csvより、publicLeaderboard用のtestデータをinput

# test_stock_prices_df = pd.read_csv('./data/supplemental_files/stock_prices.csv')
# test_stock_prices_df = test_stock_price_preprocessing_for_eval(test_stock_prices_df)

def PL_nonTimeSeriesAPI(submission):
    """
    2021/12/6から2022/2/28の期間のscoreを計算する。
    （ただし、TimeSeriesAPIには則っていないので、逐次的には計算していない。）
    
    Inputs
    ----------
    submission : pd.DataFrame
        sample_submissionに則った形式のデータ
    
    return
    ----------
    score : float
        期間中のscore
    """
    
    def calc_spread_return_per_day(day_pred, day_test):
        """当該日付分だけ抽出した状態で、当該日付のR_dayを計算"""

        day_test = day_test.merge(right=day_pred, on='SecuritiesCode')
        day_test.sort_values('Rank', inplace=True)

        day_S_up = (day_test[:200]['Target'].values * weights).sum() / weights_mean
        day_S_down = (day_test[-200:]['Target'].values[::-1] * weights).sum() / weights_mean
        day_R = day_S_up - day_S_down
        return day_R
    
    start = datetime(2021, 12, 6)
    end = datetime(2022, 2, 28)
    R = []
    for date in daterange(start, end):
        day_test = test_stock_prices_df.loc[test_df['Date'] == date].drop('Date', axis=1)
        day_pred = submission.loc[sub['Date'] == date].drop('Date', axis=1)
        if day_test.shape[0] == 0:
            continue

        R.append(calc_spread_return_per_day(day_pred, day_test))
    R = np.array(R)
    score = R.mean()/ R.std()
    
    return score